In [26]:
from azureml.core import Workspace, Experiment

ws = Workspace.get(name="quick-starts-ws-135073")
exp = Experiment(workspace=ws, name="quick-starts-ws-135073")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

Workspace name: quick-starts-ws-135073
Azure region: southcentralus
Subscription id: cdbe0b43-92a0-4715-838a-f2648cc7ad21
Resource group: aml-quickstarts-135073


In [27]:
from azureml.core.compute import ComputeTarget, AmlCompute

# TODO: Create compute cluster
# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.
from azureml.core.compute_target import ComputeTargetException

# Choose a name for your CPU cluster
cpu_cluster_name = "cpucluster"

# Verify that cluster does not exist already
try:
    cpu_cluster = ComputeTarget(workspace=ws, name=cpu_cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2',
                                                           max_nodes=4)
    cpu_cluster = ComputeTarget.create(ws, cpu_cluster_name, compute_config)

cpu_cluster.wait_for_completion(show_output=True)

CreatingAmlCompute is getting created. Consider calling wait_for_completion() first

AmlCompute is getting created. Consider calling wait_for_completion() first


Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


In [28]:
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import uniform
from azureml.core.script_run_config import ScriptRunConfig
from azureml.train.hyperdrive import choice
import os

# Specify parameter sampler
ps = RandomParameterSampling({
       "--C": uniform(0.0001, 1),
       "--max_iter": choice(80, 100, 120, 140, 180)
   })

# Specify a Policy
policy = BanditPolicy(evaluation_interval=100, slack_factor=0.2, slack_amount=None, delay_evaluation=200)

if "training" not in os.listdir():
    os.mkdir("./training")

# Create a SKLearn estimator for use with train.py
est = SKLearn(source_directory=".", compute_target="cpucluster", 
                vm_size="STANDARD_DS2_V2", vm_priority='lowpriority', entry_script='train.py')

# Create a HyperDriveConfig using the estimator, hyperparameter sampler, and policy.
hyperdrive_config = HyperDriveConfig(ps, primary_metric_name='Accuracy', primary_metric_goal=PrimaryMetricGoal.MAXIMIZE, max_total_runs=4, max_concurrent_runs=4, policy=policy, estimator=est)


In [29]:
%%writefile conda_dependencies.yml
dependencies:
- python=3.6.2
- scikit-learn
- pip:
  - azureml-defaults

Overwriting conda_dependencies.yml


In [30]:
from azureml.widgets import RunDetails

# Submit your hyperdrive run to the experiment and show run details with the widget.
# run a trial from the train.py code in your current directory
hyperdrive_run = exp.submit(hyperdrive_config)

RunDetails(hyperdrive_run).show()    



_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

In [6]:
import joblib
# Get your best run and save the model from that run.
model = hyperdrive_run.get_best_run_by_primary_metric(include_failed=True, include_canceled=True, include_resume_from_runs=True)
hyperdrive_run.wait_for_completion(show_output=True)

best_run = hyperdrive_run.get_metrics()           
joblib.dump(value=best_run, filename="./outputs/best_run.joblib") 


RunId: HD_8cfd41da-0f2d-496a-8855-0236ce845011
Web View: https://ml.azure.com/experiments/quick-starts-ws-135073/runs/HD_8cfd41da-0f2d-496a-8855-0236ce845011?wsid=/subscriptions/cdbe0b43-92a0-4715-838a-f2648cc7ad21/resourcegroups/aml-quickstarts-135073/workspaces/quick-starts-ws-135073

Execution Summary
RunId: HD_8cfd41da-0f2d-496a-8855-0236ce845011
Web View: https://ml.azure.com/experiments/quick-starts-ws-135073/runs/HD_8cfd41da-0f2d-496a-8855-0236ce845011?wsid=/subscriptions/cdbe0b43-92a0-4715-838a-f2648cc7ad21/resourcegroups/aml-quickstarts-135073/workspaces/quick-starts-ws-135073



['./outputs/best_run.joblib']

In [7]:
joblib.load('./outputs/best_run.joblib')

{'HD_8cfd41da-0f2d-496a-8855-0236ce845011_0': {'Regularization Strength:': 0.4840389338857808,
  'Max iterations:': 100,
  'Accuracy': 0.9100606952363436},
 'HD_8cfd41da-0f2d-496a-8855-0236ce845011_1': {'Regularization Strength:': 0.95120350202593,
  'Max iterations:': 120,
  'Accuracy': 0.910152657715652},
 'HD_8cfd41da-0f2d-496a-8855-0236ce845011_2': {'Regularization Strength:': 0.4250739263501935,
  'Max iterations:': 120,
  'Accuracy': 0.9100606952363436},
 'HD_8cfd41da-0f2d-496a-8855-0236ce845011_3': {'Regularization Strength:': 0.4705719798485419,
  'Max iterations:': 80,
  'Accuracy': 0.9100606952363436}}

#  AUTO ML  #

In [8]:
from azureml.data.dataset_factory import TabularDatasetFactory
# Create TabularDataset using TabularDatasetFactory
# Data is available at: 
# "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"
csv_file = 'https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv'
data = TabularDatasetFactory.from_delimited_files(csv_file)

In [9]:
from train import clean_data
from sklearn.model_selection import train_test_split
import pandas as pd
x, y = clean_data(data)
# Use the clean_data function to clean your data.
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.33, random_state=42)
train_data = pd.concat([x_train, y_train], axis = 1)


In [10]:
from azureml.train.automl import AutoMLConfig

# Set parameters for AutoMLConfig
# NOTE: DO NOT CHANGE THE experiment_timeout_minutes PARAMETER OR YOUR INSTANCE WILL TIME OUT.
# If you wish to run the experiment longer, you will need to run this notebook in your own
# Azure tenant, which will incur personal costs.
automl_config = AutoMLConfig(
    experiment_timeout_minutes=30,
    task='classification',
    primary_metric='accuracy',
    training_data=train_data,
    label_column_name='y',
    n_cross_validations=2)

In [11]:
# Submit your automl run

run = exp.submit(automl_config, show_output=True)
### YOUR CODE HERE ###    

No run_configuration provided, running on local with default configuration
Running on local machine
Parent Run ID: AutoML_46aa3c5f-a6bf-4424-a6fe-5198d3605413

Current status: DatasetEvaluation. Gathering dataset statistics.
Current status: FeaturesGeneration. Generating features for the dataset.
Current status: DatasetFeaturization. Beginning to fit featurizers and featurize the dataset.
Current status: DatasetFeaturizationCompleted. Completed fit featurizers and featurizing the dataset.
Current status: DatasetBalancing. Performing class balancing sweeping
Current status: DatasetCrossValidationSplit. Generating individually featurized CV splits.

****************************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       ALERTED
DESCRIPTION:  To decrease model bias, please cancel the current run and fix balancing problem.
              Learn more about imbalanced data: https://aka.ms/Auto

In [12]:
# Retrieve and save your best automl model.
best_run, best_model = run.get_output()
best_run.register_model(model_name='best_automl_model.pkl', model_path = './outputs/')
### YOUR CODE HERE ###

Model(workspace=Workspace.create(name='quick-starts-ws-135073', subscription_id='cdbe0b43-92a0-4715-838a-f2648cc7ad21', resource_group='aml-quickstarts-135073'), name=best_automl_model.pkl, id=best_automl_model.pkl:1, version=1, tags={}, properties={})

In [13]:
best_run.get_tags() 

{'ensembled_iterations': '[1, 45, 35, 42, 32, 44, 46, 43, 51]',
 'ensembled_algorithms': "['XGBoostClassifier', 'XGBoostClassifier', 'XGBoostClassifier', 'XGBoostClassifier', 'XGBoostClassifier', 'LightGBM', 'LightGBM', 'XGBoostClassifier', 'XGBoostClassifier']",
 'ensemble_weights': '[0.14285714285714285, 0.07142857142857142, 0.07142857142857142, 0.2857142857142857, 0.14285714285714285, 0.07142857142857142, 0.07142857142857142, 0.07142857142857142, 0.07142857142857142]',
 'best_individual_pipeline_score': '0.9142054720057982',
 'best_individual_iteration': '1',
 'model_explanation': 'True'}

In [14]:
best_run_metrics = best_run.get_metrics() 
for metric_name in best_run_metrics:
     metric = best_run_metrics[metric_name]
     print(metric_name, metric)
 

recall_score_micro 0.9154738177206015
log_loss 0.177909130857201
AUC_micro 0.9802210435406047
AUC_macro 0.9457605645070959
precision_score_weighted 0.9091326361647714
average_precision_score_macro 0.8209161596176471
average_precision_score_micro 0.9810499346456378
balanced_accuracy 0.7448210394288107
matthews_correlation 0.5393187751555846
AUC_weighted 0.9457605645070959
f1_score_macro 0.7673865382099132
precision_score_micro 0.9154738177206015
precision_score_macro 0.7970210945866707
recall_score_weighted 0.9154738177206015
norm_macro_recall 0.48964207885762145
recall_score_macro 0.7448210394288107
weighted_accuracy 0.9578586743593104
average_precision_score_weighted 0.9544016375169213
accuracy 0.9154738177206015
f1_score_micro 0.9154738177206015
f1_score_weighted 0.9114011683696854
accuracy_table aml://artifactId/ExperimentRun/dcid.AutoML_46aa3c5f-a6bf-4424-a6fe-5198d3605413_54/accuracy_table
confusion_matrix aml://artifactId/ExperimentRun/dcid.AutoML_46aa3c5f-a6bf-4424-a6fe-5198d360

In [16]:
from pprint import pprint

def print_model(best_model, prefix=""):
    for step in best_model.steps:
        print(prefix + step[0])
        if hasattr(step[1], 'estimators') and hasattr(step[1], 'weights'):
            pprint({'estimators': list(
                e[0] for e in step[1].estimators), 'weights': step[1].weights})
            print()
            for estimator in step[1].estimators:
                print_model(estimator[1], estimator[0] + ' - ')
        else:
            pprint(step[1].get_params())
            print()

print_model(best_model)

datatransformer
{'enable_dnn': None,
 'enable_feature_sweeping': None,
 'feature_sweeping_config': None,
 'feature_sweeping_timeout': None,
 'featurization_config': None,
 'force_text_dnn': None,
 'is_cross_validation': None,
 'is_onnx_compatible': None,
 'logger': None,
 'observer': None,
 'task': None,
 'working_dir': None}

prefittedsoftvotingclassifier
{'estimators': ['1', '45', '35', '42', '32', '44', '46', '43', '51'],
 'weights': [0.14285714285714285,
             0.07142857142857142,
             0.07142857142857142,
             0.2857142857142857,
             0.14285714285714285,
             0.07142857142857142,
             0.07142857142857142,
             0.07142857142857142,
             0.07142857142857142]}

1 - maxabsscaler
{'copy': True}

1 - xgboostclassifier
{'base_score': 0.5,
 'booster': 'gbtree',
 'colsample_bylevel': 1,
 'colsample_bynode': 1,
 'colsample_bytree': 1,
 'gamma': 0,
 'learning_rate': 0.1,
 'max_delta_step': 0,
 'max_depth': 3,
 'min_child_weight'

In [17]:
cpu_cluster.delete() 

Current provisioning state of AmlCompute is "Deleting"

